In [ ]:
from fastai import *        # Quick accesss to most common functionality
from fastai.text import *   # Quick accesss to NLP functionality

In [ ]:
path = untar_data(URLs.IMDB_SAMPLE)
path

PosixPath('/home/ubuntu/.fastai/data/imdb_sample')

In [ ]:
data_lm = (TextList.from_csv(path, 'texts.csv', create_func=None, col='text')#, processor=TextProcessor())
                   .random_split_by_pct()
                   .label_for_lm()
                   .databunch())

In [ ]:
data_lm.show_batch()

idx,text
0,"xxfld 1 first of all , in defense of xxup joan xxup fontaine , it must be said that ginger rogers would have been terribly miscast as xxunk , the young british lady who has the title role . fontaine makes a xxunk picture as the heroine here , but her acting xxunk shows badly and her dancing is better left xxunk . fortunately , she went on to better things . \n\n but here it 's xxup fred xxup astaire , xxup george xxup burns and xxup gracie xxup allen who get the top"
1,"only store . xxfld 1 i think james cameron might be becoming my favorite director because this is my second review of his movies . anyway , everyone xxunk the xxup xxunk titanic . it was big , fast , and "" xxunk "" ... until april xxunk . it was all over the news and one of the biggest xxunk ever . well james cameron decided to make a movie out of it but star two fictional characters to be in the spotlight instead of the ship . well , onto the main review"
2,horrible . several scenes were dragged out way too long in an attempt at suspense and the effects were non - existent . the attack by the xxunk in the xxunk should have been completely xxunk from the final cut and every attempt to bring life to the xxunk was obvious with stick pokes and xxunk . i also could n't help but think the budget did n't allow them to xxunk the house so they kept making references to the xxunk and that all the things in xxunk should be coming soon . honestly
3,"films . xxunk is a breath of fresh air , a film with so many xxunk that one viewing is not enough to understand or appreciate this outstanding film . xxunk von xxunk shows that old styles of filming can produce marvellous cinema and build drama and tension . the back xxunk effect he uses during the film xxunk and enhances the characters , and the focus of the conversation they are having . other effects he uses such as the colour and black and white in one scene much like hitchcock and the girl"
4,"when it is given a chance . she xxunk deserved her oscar . \n\n this movie is in an epic in the most xxunk proportions . all parts xxunk , necessary and perfect . xxunk may walk away if they wish , but they would be xxunk a storytelling which stays with one a lifetime . \n\n xxup the xxup good xxup earth xxunk one in ways that one does not expect . but all will not come from it with the joy that i did . but i can only hope that this film"
5,"by taking on many roles for his project . of course this low budget film could use better editing xxunk and more special effects for the gun scenes , but the music keeps this script moving . although this film has it 's share of problems , such as continuity , i must say that i would rent the director 's next movie . if your a film student , you could learn a few things from the director 's commentary . xxfld 1 nagra comes from conservative indian family that is n't exactly into"
6,"is for you . positive comments : interesting cinematography at times , wasted on the other elements . very realistic gore ; again , wasted . but the intestines scene is classic . i agree with the xxunk xxunk xxunk the fake accent problem . xxfld 1 this is one horror movie based tv show that gets it right . friday the 13th the series had no connection to the movies . xxunk the legacy : i 'm not so sure . it may have been loosely connected to the movies . it feels like"
7,"movie . a great debut behind the camera . i regret the xxunk title of this movie : ' a xxunk do xxunk ' ( the devil 's hand ' ) . if at least it were ' the god 's hand ' , it might be acceptable . but calling this movie as ' the devil 's hand ' is indeed ridiculous . xxunk xxunk , the screenwriter , did not deserve such a lack of respect from the xxunk xxunk . this film is highly recommended . my vote is xxunk . \n\n"
8,"though no doubt doing what she was directed to do , had no xxunk bearing , and all the girls seemed to lose the xxunk of their characters - also affected by xxunk cuts , which not only took away the female characters alr

In [ ]:
len(data_lm.train_ds.vocab.itos), len(data_lm.valid_ds.vocab.itos)

(6108, 6108)

In [ ]:
data_clas = (TextList.from_csv(path, 'texts.csv', create_func=None, col='text', vocab=data_lm.vocab)
                     .random_split_by_pct()
                     .label_from_df(cols=0)
                     .databunch())

In [ ]:
data_clas.show_batch()

text,label
"xxfld 1 xxunk victor xxunk : a review \n\n you know , xxunk victor xxunk is like sticking your hands into a big , xxunk xxunk of xxunk . it 's warm and xxunk , but you 're not sure if it feels right . try as i might ,",negative
"xxfld 1 now that che(2008 ) has finished its relatively short australian cinema run ( extremely limited xxunk screen in xxunk , after xxunk ) , i can xxunk join both xxunk of "" at the movies "" in taking steven soderbergh to task . \n\n it 's usually satisfying",negative
"xxfld 1 many xxunk that this is n't just a classic due to the fact that it 's the first 3d game , or even the first xxunk - up . it 's also one of the first xxunk games , one of the xxunk definitely the first ) truly",positive
xxfld 1 this film sat on my xxunk for weeks before i watched it . i xxunk a self - indulgent xxunk flick about relationships gone bad . i was wrong ; this was an xxunk xxunk into the xxunk - up xxunk of new xxunk . \n\n the format,positive
"xxfld 1 \n\n i 'm sure things did n't exactly go the same way in the real life of xxunk xxunk as they did in the film adaptation of his book , rocket boys , but the movie "" xxunk sky "" ( an xxunk of the book 's title",positive


In [ ]:
learn = language_model_learner(data_lm, pretrained_model=URLs.WT103)

In [ ]:
learn.predict('Jeremy Howard is', 4)

Total time: 00:00



'Jeremy Howard is xxunk xxunk ( xxunk'

In [ ]:
learn = text_classifier_learner(data_clas)

In [ ]:
learn.predict('I really liked this movie!')

('negative', tensor(0), tensor([0.5415, 0.4585]))